In [ ]:
# load dependencies and allow interactivity
%matplotlib widget

import rioxarray as rxr
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
import numpy as np

In [ ]:
data_direc = '/home/etboud/projects/data/planet/DNBC/'

BS = '/home/etboud/projects/data/basins/DLNY/DLNY_4326.geojson'
basin = gpd.read_file(BS)
basin=basin.to_crs('EPSG:32611')

subdirecs = sorted([d for d in glob.glob(data_direc + str(focus_year) + '*') if os.path.isdir(d)])
print('length of data record: ',len(subdirecs))

In [ ]:
#### functions
def calc_rgb(ds):
    # Selecting RGB bands
    blue_band = ds.isel(band=0)
    green_band = ds.isel(band=1)
    red_band = ds.isel(band=2)
    nir_band = ds.isel(band=3)
    
    # normalize to help visual understanding
    maxval = green_band.max().values
    minval = green_band.min().values
    red_norm = (red_band - minval) / (maxval - minval)
    green_norm = (green_band - minval) / (maxval - minval)
    blue_norm = (blue_band - minval) / (maxval - minval)
    green_norm = green_norm.where(red_norm <= 1,1)
    blue_norm = blue_norm.where(red_norm <= 1,1)
    red_norm = red_norm.where(red_norm <= 1,1)

    # port to numpy
    red_band = red_band.values
    green_band = green_band.values
    blue_band = blue_band.values
    nir_band = nir_band.values
    
    # Stack normalized bands to create RGB image
    rgb_image = np.stack([red_norm, green_norm, blue_norm], axis=-1)
    return red_band,green_band,blue_band,nir_band,rgb_image



In [ ]:
for direcCount, direcc in enumerate(subdirecs):  # Adjusted to start at the 44th .tif, [89:], start=88 (this might throw off indexing TODO: CHECK THIS)
    try:
        fname = glob.glob(direcc + '/*/PSScene/*SR_clip.tif')[0]
        ds = rxr.open_rasterio(fname)
        red_band, green_band, blue_band, nir_band, rgb_image = calc_rgb(ds)

        fg, ax = plt.subplots(figsize=(4, 4))
        ax.imshow(rgb_image, cmap='gray')
        ax.set_title(f'{direcCount}, {fname.split("/")[-1].split("_")[0]}')

    except:
        print('bad data')

In [ ]:
# run this to determine what set of 10 images to select
# view all of the plots to make the set of 10 selection
# only for visualization, so comment out if not using

# for direcCount,direcc in enumerate(subdirecs):
#     try:
#         fname = glob.glob(direcc+'/*/PSScene/*SR_clip.tif')[0]
#         ds = rxr.open_rasterio(fname)
#         red_band, green_band, blue_band, nir_band, rgb_image = calc_rgb(ds)

#         fg,ax = plt.subplots(figsize=(4,4))
#         ax.imshow(rgb_image,cmap='gray')
#         ax.set_title(direcCount)
        
#     except:
#         print('bad data')ps_tif = '/home/etboud/projects/data/planet/DLNY/20230629_175448_06_24ce/88c4e89b-3759-43f1-8d5d-9daada6cab22/PSScene/20230629_175448_06_24ce_3B_AnalyticMS_SR_clip.tif'


In [ ]:
ps_tif = '/home/etboud/projects/data/planet/DLNY/20230629_175448_06_24ce/88c4e89b-3759-43f1-8d5d-9daada6cab22/PSScene/20230629_175448_06_24ce_3B_AnalyticMS_SR_clip.tif'

# Open the raster dataset
with rxr.open_rasterio(ps_tif, masked=True) as ds:
    ds_clipped = ds.rio.clip(basin.geometry)
    # Calculate RGB and NIR bands
    red_band, green_band, blue_band, nir_band, rgb_image = calc_rgb(ds_clipped)

    # Initialize ps_points list
    ps_points = []

    # Get dimensions of the image
    height, width = red_band.shape

    # Iterate over all pixel coordinates
    for y in range(height):
        for x in range(width):
            ps_points.append((x, y, red_band[y, x], green_band[y, x], blue_band[y, x], nir_band[y, x],
                                rgb_image[y, x, 0], rgb_image[y, x, 1], rgb_image[y, x, 2]))

# Create dataframe for PS data
ps_df = pd.DataFrame(ps_points, columns=['x', 'y','r','g','b','nir','r_norm','g_norm','b_norm'])

In [ ]:
# Paths to files
aso_tif = '/home/etboud/projects/data/aso/binary/ASO_Tuolumne_2023Jun26-27_snowdepth_3m_binary.tif'

# Open and clip the raster dataset
with rxr.open_rasterio(aso_tif, masked=True) as ds:
    ds_clipped = ds.rio.clip(basin.geometry, drop=False)

    # Initialize aso_points list
    aso_points = []

    # Get dimensions of the clipped image
    height, width = ds_clipped.shape[1], ds_clipped.shape[2]

    # Iterate over all pixel coordinates
    for y in range(height):
        for x in range(width):
            aso_points.append((x, y, ds_clipped.values[0, y, x]))

# Create dataframe for aso label
aso_df = pd.DataFrame(aso_points, columns=['x', 'y', 'label'])

In [ ]:
# Merge the two dataframes on the pixel counts
merged_df = pd.merge(aso_df, ps_df, on=['x', 'y'])

In [ ]:
df = pd.read_csv('/home/etboud/projects/unorganized/aso_classified_2023_class01.csv')

In [ ]:
df.dropna()